In [1]:
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import locale

# Set locale for Indian number formatting (lakh, crore)
locale.setlocale(locale.LC_ALL, 'en_IN.UTF-8')

def sip_calculator(monthly_investment, annual_rate, years, yearly_increase):
    # Convert annual interest rate to a monthly rate
    monthly_rate = annual_rate / 12 / 100
    
    total_investment_value = 0
    current_monthly_investment = monthly_investment
    total_investment = 0  # To track the total amount invested

    # Loop through each year
    for year in range(1, years + 1):
        for month in range(12):
            months_left = (years * 12) - ((year - 1) * 12 + month)
            total_investment_value += current_monthly_investment * ((1 + monthly_rate) ** months_left)

        # Track total investment made
        total_investment += current_monthly_investment * 12

        # Increase the monthly SIP at the end of each year
        current_monthly_investment += current_monthly_investment * (yearly_increase / 100)

    # Calculate the compounded returns earned
    compounded_returns = total_investment_value - total_investment

    return total_investment_value, total_investment, compounded_returns

def plot_donut_chart(canvas_frame, total_investment, compounded_returns):
    # Clear the frame for new plot
    for widget in canvas_frame.winfo_children():
        widget.destroy()

    # Data for the donut chart (absolute values)
    labels = ['Investments (₹{})'.format(locale.format_string('%d', total_investment, grouping=True)),
              'Compounded Returns (₹{})'.format(locale.format_string('%d', compounded_returns, grouping=True))]
    values = [total_investment, compounded_returns]

    # Create a donut chart
    fig, ax = plt.subplots(figsize=(6, 6))
    wedges, texts, autotexts = ax.pie(values, labels=labels, startangle=90, wedgeprops={'linewidth': 1, 'edgecolor': 'white'}, 
                                      autopct=lambda p: '{:.0f}'.format(p * sum(values) / 100))
    
    # Create a circle at the center to make it a donut
    center_circle = plt.Circle((0, 0), 0.70, color='white')
    fig.gca().add_artist(center_circle)

    # Equal aspect ratio ensures the pie is drawn as a circle
    ax.axis('equal')

    # Embed the plot into Tkinter using FigureCanvasTkAgg
    canvas = FigureCanvasTkAgg(fig, master=canvas_frame)
    canvas.draw()
    canvas.get_tk_widget().pack()

def calculate_sip():
    try:
        monthly_investment = float(entry_monthly_investment.get())
        annual_rate = float(entry_annual_rate.get())
        years = int(entry_years.get())
        yearly_increase = float(entry_yearly_increase.get())
        
        # Calculate the SIP future value and breakdown
        total_value, total_investment, compounded_returns = sip_calculator(monthly_investment, annual_rate, years, yearly_increase)

        # Display the results in rupees format
        result_text.set(f"Future Value: ₹{locale.format_string('%d', total_value, grouping=True)}\n"
                        f"Total Investment: ₹{locale.format_string('%d', total_investment, grouping=True)}\n"
                        f"Compounded Returns: ₹{locale.format_string('%d', compounded_returns, grouping=True)}")

        # Show donut chart in the canvas
        plot_donut_chart(canvas_frame, total_investment, compounded_returns)

    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid numeric values.")

# Create the main application window
app = tk.Tk()
app.title("SIP Calculator")
app.geometry("600x600")
app.configure(bg="white")  # Set background to white

# Green theme color for widgets
theme_color = "#4CAF50"

# Labels and Entry Fields
tk.Label(app, text="SIP Calculator", font=("Arial", 20, "bold"), bg="white", fg=theme_color).pack(pady=10)

tk.Label(app, text="Monthly Investment (₹):", bg="white", fg=theme_color).pack()
entry_monthly_investment = tk.Entry(app)
entry_monthly_investment.pack()

tk.Label(app, text="Annual Interest Rate (%):", bg="white", fg=theme_color).pack()
entry_annual_rate = tk.Entry(app)
entry_annual_rate.pack()

tk.Label(app, text="Investment Period (Years):", bg="white", fg=theme_color).pack()
entry_years = tk.Entry(app)
entry_years.pack()

tk.Label(app, text="Yearly Increase in SIP (%):", bg="white", fg=theme_color).pack()
entry_yearly_increase = tk.Entry(app)
entry_yearly_increase.pack()

# Button to calculate SIP
calculate_button = tk.Button(app, text="Calculate SIP", bg=theme_color, fg="white", command=calculate_sip)
calculate_button.pack(pady=10)

# Result display
result_text = tk.StringVar()
result_label = tk.Label(app, textvariable=result_text, bg="white", fg=theme_color, font=("Arial", 12))
result_label.pack(pady=10)

# Create a frame for the canvas (where the donut chart will be shown)
canvas_frame = tk.Frame(app, bg="white")
canvas_frame.pack(pady=10)

# Start the main loop
app.mainloop()
